In [1]:
import torch 
import torchvision 

import torch.nn as nn 
from torchvision import transforms

import matplotlib.pyplot as plt
from torchinfo import summary

In [2]:
# device agnostic code 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
# set seeds 

def set_seeds(seed:int = 42):
    # set the seed for general torch operations 
    torch.manual_seed(seed)
    # set the seed for CUDA torch operations 
    torch.cuda.manual_seed(seed)

In [4]:
# get the data (We have already downoad the data)

import os 
from pathlib import Path 

data_path = Path('DATA')
image_path = data_path/"pizza_steak_sushi"

# set up train and test directories 
train_dir = image_path/'train'
test_dir = image_path/'test'

# setup pretrined weights 
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

# get the transforms from weights 
automatic_transforms = weights.transforms()

# create data loaders 
from going_modular import data_setup
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir, test_dir=test_dir,transform=automatic_transforms, batch_size=32)


In [5]:
# creating model with trnsfer learning  
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

# freeze all base layers 
for param in model.features.parameters():
    param.requires_grad = False 
    
# and update the classifier head to suit our problem 
model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names),bias=True)).to(device)

In [6]:
# print the model summary 
from torchinfo import summary
summary(model=model,input_size=(32,3,224,224), col_names=['input_size','output_size','num_params','trainable'],
        col_width=20,row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [7]:
# define loss and optimizer 
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
import torch.utils
from going_modular.engine import train_step, test_step
from typing import Dict, List 
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int,
          device:torch.device) -> Dict[str,List]:
    # create empty results dictionary 
    results = {"train_loss":[],"train_acc":[],
               "test_loss":[],"test_acc":[]}
    
    # loop through training and testing steps for a number of epochs 
    for epoch in range(epochs):
        train_loss,train_acc = train_step(model=model, dataloader=train_dataloader,
                                          loss_fn=loss_fn,optimizer=optimizer,device=device)
        test_loss,test_acc = test_step(model=model,dataloader=test_dataloader,
                                       loss_fn=loss_fn,device=device)
        
        # print what is happening 
        print(f"Epoch:{epoch+1} | train_loss:{train_loss:.4f} | train_acc:{train_acc:.4f}\
            | test_loss:{test_loss:.4f} | test_acc:{test_acc:.4f}")
        
        # save the results to dictionary 
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        # experimental tracking 
        writer.add_scalars(main_tag="Loss",tag_scalar_dict={"train_loss":train_loss, "test_loss":test_loss},global_step=epoch)
        writer.add_scalars(main_tag="Accuracy",tag_scalar_dict={"train_acc":train_acc,"test_acc":test_acc},global_step=epoch)
        writer.add_graph(model=model,input_to_model=torch.randn(32,3,224,224).to(device))
        
    writer.close()
    
    return results 

In [9]:
# train the model 
set_seeds()
results = train(model=model,train_dataloader=train_dataloader,test_dataloader=test_dataloader,
                optimizer=optimizer,loss_fn=loss_fn,epochs=5,device=device)

Epoch:1 | train_loss:1.1141 | train_acc:0.3867            | test_loss:0.9437 | test_acc:0.5786
Epoch:2 | train_loss:0.9072 | train_acc:0.6484            | test_loss:0.8260 | test_acc:0.7936
Epoch:3 | train_loss:0.8040 | train_acc:0.7500            | test_loss:0.7084 | test_acc:0.8542
Epoch:4 | train_loss:0.6661 | train_acc:0.8867            | test_loss:0.6983 | test_acc:0.8144
Epoch:5 | train_loss:0.6986 | train_acc:0.7266            | test_loss:0.7046 | test_acc:0.7538


In [10]:
# create custom summary writer instance 

from datetime import datetime
import os

def create_writer(experiment_name:str,model_name:str, extra:str=None)-> torch.utils.tensorboard.writer.SummaryWriter():
    timestamp = datetime.now().strftime("%Y-%m-%d")
    
    if extra:
        log_dir = os.path.join("runs",timestamp,experiment_name,model_name,extra)
    else:
        log_dir = os.path.join("runs",timestamp,experiment_name,model_name)
        
    print(f"[INFO] created SummaryWriter, saving to:{log_dir}")
    
    return SummaryWriter(log_dir=log_dir)

In [11]:
# create example writer 
example_writer = create_writer(experiment_name="data_10_percent",model_name="effnetb0",extra="5_epochs")

[INFO] created SummaryWriter, saving to:runs/2024-06-12/data_10_percent/effnetb0/5_epochs


In [26]:
# let's include write function inside the train function 

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int,
          device:torch.device,
          writer:torch.utils.tensorboard.writer.SummaryWriter) -> Dict[str,List]:
    # create empty results dictionary 
    results = {"train_loss":[],"train_acc":[],
               "test_loss":[],"test_acc":[]}
    
    # loop through training and testing steps for a number of epochs 
    for epoch in range(epochs):
        train_loss,train_acc = train_step(model=model, dataloader=train_dataloader,
                                          loss_fn=loss_fn,optimizer=optimizer,device=device)
        test_loss,test_acc = test_step(model=model,dataloader=test_dataloader,
                                       loss_fn=loss_fn,device=device)
        
        # print what is happening 
        print(f"Epoch:{epoch+1} | train_loss:{train_loss:.4f} | train_acc:{train_acc:.4f}\
            | test_loss:{test_loss:.4f} | test_acc:{test_acc:.4f}")
        
        # save the results to dictionary 
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        # experimental tracking 
        if writer:
            writer.add_scalars(main_tag="Loss",tag_scalar_dict={"train_loss":train_loss, "test_loss":test_loss},global_step=epoch)
            writer.add_scalars(main_tag="Accuracy",tag_scalar_dict={"train_acc":train_acc,"test_acc":test_acc},global_step=epoch)
            writer.add_graph(model=model,input_to_model=torch.randn(32,3,224,224).to(device))
            
            writer.close()
        else:
            pass
        
    return results 

In [ ]:
# let's run with 20% data, with 10 epochs in model effnet_0 
epochs = 10 
train(model=model,train_dataloader=train_dataloader,test_dataloader=test_dataloader,
      optimizer=optimizer,loss_fn=loss_fn,epochs=epochs,device=device,
      writer= create_writer(experiment_name='20_percent_data',model_name="effnetb0",extra=f"{epochs}_epochs"))